In [1]:
import pandas as pd

oof_pred = pd.read_csv('./data_mms/fold_preds.csv')
oof_pred.drop(['age','fold'], axis=1, inplace=True)
oof_pred.columns = ['user_id', 'is_male_proba', 'age_bucket_1_proba', 'age_bucket_2_proba',
       'age_bucket_3_proba', 'age_bucket_4_proba', 'age_bucket_5_proba',
       'age_bucket_6_proba']

test_pred = pd.read_csv('./data_mms/test_preds.csv')
test_pred.drop(['age'], axis=1, inplace=True)
test_pred.columns = ['user_id', 'is_male_proba', 'age_bucket_1_proba', 'age_bucket_2_proba',
       'age_bucket_3_proba', 'age_bucket_4_proba', 'age_bucket_5_proba',
       'age_bucket_6_proba']

tree_pred = pd.concat([oof_pred,test_pred], copy=False, ignore_index=True).sort_values('user_id').reset_index(drop=True)
tree_pred_cols = ['is_male_proba', 'age_bucket_1_proba', 'age_bucket_2_proba',
       'age_bucket_3_proba', 'age_bucket_4_proba', 'age_bucket_5_proba',
       'age_bucket_6_proba']

oof_pred = pd.read_csv('./data_mms/fold_preds-clean.csv')
oof_pred.drop(['age','fold'], axis=1, inplace=True)
oof_pred.columns = ['user_id', 'is_male_proba_clean', 'age_bucket_1_proba_clean', 'age_bucket_2_proba_clean',
       'age_bucket_3_proba_clean', 'age_bucket_4_proba_clean', 'age_bucket_5_proba_clean',
       'age_bucket_6_proba_clean']

test_pred = pd.read_csv('./data_mms/test_preds-clean.csv')
test_pred.drop(['age'], axis=1, inplace=True)
test_pred.columns = ['user_id', 'is_male_proba_clean', 'age_bucket_1_proba_clean', 'age_bucket_2_proba_clean',
       'age_bucket_3_proba_clean', 'age_bucket_4_proba_clean', 'age_bucket_5_proba_clean',
       'age_bucket_6_proba_clean']

tree_pred_clean = pd.concat([oof_pred,test_pred], copy=False, ignore_index=True).sort_values('user_id').reset_index(drop=True)
tree_pred_clean_cols = ['is_male_proba_clean', 'age_bucket_1_proba_clean', 'age_bucket_2_proba_clean',
       'age_bucket_3_proba_clean', 'age_bucket_4_proba_clean', 'age_bucket_5_proba_clean',
       'age_bucket_6_proba_clean']

tree_pred_clean = tree_pred_clean.groupby('user_id').agg('first').reset_index()

oof_pred = pd.read_csv('./data_mms/fold_preds-top.csv')
oof_pred.drop(['age','fold'], axis=1, inplace=True)
oof_pred.columns = ['user_id', 'is_male_proba_top', 'age_bucket_1_proba_top', 'age_bucket_2_proba_top',
       'age_bucket_3_proba_top', 'age_bucket_4_proba_top', 'age_bucket_5_proba_top',
       'age_bucket_6_proba_top']

test_pred = pd.read_csv('./data_mms/test_preds-top.csv')
test_pred.drop(['age'], axis=1, inplace=True)
test_pred.columns = ['user_id', 'is_male_proba_top', 'age_bucket_1_proba_top', 'age_bucket_2_proba_top',
       'age_bucket_3_proba_top', 'age_bucket_4_proba_top', 'age_bucket_5_proba_top',
       'age_bucket_6_proba_top']

tree_pred_top = pd.concat([oof_pred,test_pred], copy=False, ignore_index=True).sort_values('user_id').reset_index(drop=True)
tree_pred_top_cols = ['is_male_proba_top', 'age_bucket_1_proba_top', 'age_bucket_2_proba_top',
       'age_bucket_3_proba_top', 'age_bucket_4_proba_top', 'age_bucket_5_proba_top',
       'age_bucket_6_proba_top']

tree_pred_top = tree_pred_top.groupby('user_id').agg('first').reset_index()

oof_pred = pd.read_csv('./data_mms/fold_preds-bigrams.csv')
oof_pred.drop(['age','fold'], axis=1, inplace=True)
oof_pred.columns = ['user_id', 'is_male_proba_bigrams', 'age_bucket_1_proba_bigrams', 'age_bucket_2_proba_bigrams',
       'age_bucket_3_proba_bigrams', 'age_bucket_4_proba_bigrams', 'age_bucket_5_proba_bigrams',
       'age_bucket_6_proba_bigrams']

test_pred = pd.read_csv('./data_mms/test_preds-bigrams.csv')
test_pred.drop(['age'], axis=1, inplace=True)
test_pred.columns = ['user_id', 'is_male_proba_bigrams', 'age_bucket_1_proba_bigrams', 'age_bucket_2_proba_bigrams',
       'age_bucket_3_proba_bigrams', 'age_bucket_4_proba_bigrams', 'age_bucket_5_proba_bigrams',
       'age_bucket_6_proba_bigrams']

tree_pred_bigrams = pd.concat([oof_pred,test_pred], copy=False, ignore_index=True).sort_values('user_id').reset_index(drop=True)
tree_pred_bigrams_cols = ['is_male_proba_bigrams', 'age_bucket_1_proba_bigrams', 'age_bucket_2_proba_bigrams',
       'age_bucket_3_proba_bigrams', 'age_bucket_4_proba_bigrams', 'age_bucket_5_proba_bigrams',
       'age_bucket_6_proba_bigrams']

tree_pred_bigrams = tree_pred_bigrams.groupby('user_id').agg('first').reset_index()

tree_pred.shape, tree_pred_clean.shape, tree_pred_top.shape, tree_pred_bigrams.shape

((414681, 8), (414681, 8), (414681, 8), (414681, 8))

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz 
from scipy.sparse import coo_matrix
import implicit
import bisect

def age_bucket(x):
    return bisect.bisect_left([25,35,45,55,65], x)

q = pd.read_parquet('./data_mms/q_f2k_20unk.pq')
df_target = pd.read_parquet('./data/public_train.pqt')
df_target['real_age'] = df_target['age']
q = q.merge(df_target[['user_id','real_age']], on='user_id', how='left')
q['age'] = q['real_age'].apply(age_bucket)
df_target = df_target.loc[~df_target.isna().any(axis=1)].reset_index(drop=True)

q = q.merge(tree_pred, on='user_id', how='left')
q = q.merge(tree_pred_clean, on='user_id', how='left')
q = q.merge(tree_pred_top, on='user_id', how='left')
q = q.merge(tree_pred_bigrams, on='user_id', how='left')
user_features = q[tree_pred_cols+tree_pred_clean_cols+tree_pred_top_cols+tree_pred_bigrams_cols].values

#user_item = load_npz('./data_mms/ui5.npz')

#model = implicit.als.AlternatingLeastSquares(factors=64)
#model.fit(user_item)

#user_features = model.user_factors.to_numpy()
#user_features2 = np.load('./data_mms/uf_svd.npy')
#user_features = np.concatenate([user_features, user_features2], axis=1)
#user_features = np.zeros((415317, 64))
print(user_features.shape)

(415317, 28)


In [2]:
q = pd.DataFrame({
    'top': tree_pred_top.is_male_proba_top,
    'bigrams': tree_pred_bigrams.is_male_proba_bigrams,
    'clean': tree_pred_clean.is_male_proba_clean,
    'first': tree_pred.is_male_proba,
})
q.corr()

,top,bigrams,clean,first
top,1.000000,0.993489,0.998200,0.997185
bigrams,0.993489,1.000000,0.993679,0.993218
clean,0.998200,0.993679,1.000000,0.995834
first,0.997185,0.993218,0.995834,1.000000


In [3]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
user_features = ss.fit_transform(user_features)

In [4]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(q.loc[~q['real_age'].isnull()].real_age.values.reshape(-1,1))
q['real_age'] = ss.transform(q['real_age'].values.reshape(-1,1))

In [5]:
#q['max_url_host'] = q['url_host'].apply(max)
#q['max_url_host_last'] = q['url_host_last'].apply(max)
#q['max_url_host'].max(), q['max_url_host_last'].max()
#(90009, 64278)

In [6]:
q.columns

Index(['user_id', 'region_name', 'city_name', 'cpe_manufacturer_name',
       'cpe_model_name', 'url_host', 'cpe_type_cd', 'cpe_model_os_type',
       'price', 'part_of_day', 'request_cnt', 'url_host_freq',
       'non_digits_cnt', 'new_url_host_freq', 'men_share', 'women_share',
       'men_bucket_share_0', 'women_bucket_share_0', 'people_in_bucket_0',
       'men_bucket_share_1', 'women_bucket_share_1', 'people_in_bucket_1',
       'men_bucket_share_2', 'women_bucket_share_2', 'people_in_bucket_2',
       'men_bucket_share_3', 'women_bucket_share_3', 'people_in_bucket_3',
       'men_bucket_share_4', 'women_bucket_share_4', 'people_in_bucket_4',
       'men_bucket_share_5', 'women_bucket_share_5', 'people_in_bucket_5',
       'men_bucket_share_6', 'women_bucket_share_6', 'people_in_bucket_6',
       'dow', 'diff_days', 'age', 'is_male', 'real_age', 'is_male_proba',
       'age_bucket_1_proba', 'age_bucket_2_proba', 'age_bucket_3_proba',
       'age_bucket_4_proba', 'age_bucket_5_prob

In [7]:
num_features = ['request_cnt', 'price', 'diff_days', 'men_share', 'women_share', 'men_bucket_share_0',
       'women_bucket_share_0', 'people_in_bucket_0', 'men_bucket_share_1',
       'women_bucket_share_1', 'people_in_bucket_1', 'men_bucket_share_2',
       'women_bucket_share_2', 'people_in_bucket_2', 'men_bucket_share_3',
       'women_bucket_share_3', 'people_in_bucket_3', 'men_bucket_share_4',
       'women_bucket_share_4', 'people_in_bucket_4', 'men_bucket_share_5',
       'women_bucket_share_5', 'people_in_bucket_5', 'men_bucket_share_6',
       'women_bucket_share_6', 'people_in_bucket_6']
cat_features = ['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
                'cpe_type_cd', 'cpe_model_os_type', 'part_of_day', 
                'url_host','dow'] #
cat_embs_size = [(81, 3), (985, 3), (37, 6), (599, 32),
                 (4, 2), (3, 2), (4, 2), 
                 (23761, 200),  (7, 3)] #

In [8]:
import torch
import numpy as np
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import gc
import tqdm
from torch.utils.data import DataLoader

class AlphaDataset(Dataset):
    def __init__(self, df, is_train = False):
        self.df = df.copy()
        self.is_train = is_train
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self,idx):
        s = self.df.iloc[idx]
        uid = s.user_id
        
        mask = np.ones(len(s['url_host']))
        cat_features_vals = np.stack(s[cat_features].values).T
        num_features_vals = np.stack(s[num_features].values).T
        user_features_vals = user_features[uid]
        if (self.is_train): # and (np.random.rand() > 0.8)
            idxs = np.arange(len(s['url_host']))
            np.random.shuffle(idxs)
            cat_features_vals = cat_features_vals[idxs]
            num_features_vals = num_features_vals[idxs]
        
        targets = [s['age'], s['is_male'],s['real_age']]
        return cat_features_vals, num_features_vals,user_features_vals, mask, targets
    

def pad_matrix(mat, max_len):
    n = mat.shape[0]
    if len(mat.shape) == 1:
        if max_len <= n:
            return torch.tensor(mat[-max_len:])
        return torch.cat([torch.tensor(mat), torch.zeros(max_len - n)])
    if max_len <= n:
        return torch.tensor(mat[-max_len:])
    return torch.cat([
        torch.tensor(mat),
        torch.zeros((max_len - n, mat.shape[1]))
    ])

def collate_batch(batch):
    #cat_features_vals, num_features_vals, user_features_vals, mask, targets = batch
    max_len = 0
    for c in batch:
        if c[0].shape[0] > max_len:
            max_len = c[0].shape[0]
            
    if max_len > 1200:
        max_len = 1200

    return torch.stack([
        pad_matrix(m[0], max_len) for m in batch
    ]), torch.stack([
        pad_matrix(m[1], max_len) for m in batch
    ]), torch.stack([
        torch.tensor(m[2]) for m in batch
    ]), torch.stack([
        pad_matrix(m[3], max_len) for m in batch
    ]), torch.stack([torch.tensor(m[4]) for m in batch])
      
#dataloader = DataLoader(ds_test, batch_size=64, collate_fn=collate_batch, shuffle=False)


In [9]:
test_df = pd.read_parquet('./data/submit_2.pqt')
test_users = test_df.user_id.unique().tolist()
df_test = q.loc[q.user_id.isin(test_users)].reset_index(drop=True)
ds_test = AlphaDataset(df_test)

In [10]:
#q['data_len'] = q['region_name'].apply(len)
#q.loc[q.data_len > 16, 'data_len'] = q.loc[q.data_len > 16, 'url_host_freq'].apply(lambda x: np.where(x < 100000)[0].shape[0])

In [11]:
q['data_len'] = q['region_name'].apply(len)

In [12]:
import numpy as np
from random import shuffle
from torch.utils.data import Sampler

class BySequenceLengthSampler(Sampler):
    def __init__(self, data_source, batch_size=64):
        ind_n_len = []
        pps = data_source['data_len'].values
        for i, p in enumerate(pps):
            ind_n_len.append( (i, p) )
        self.ind_n_len = ind_n_len
        self.bucket_boundaries = np.quantile(pps, [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8])
        self.batch_size = batch_size
        
    def __iter__(self):
        data_buckets = dict()
        # where p is the id number and seq_len is the length of this id number. 
        for p, seq_len in self.ind_n_len:
            pid = self.element_to_bucket_id(p,seq_len)
            if pid in data_buckets.keys():
                data_buckets[pid].append(p)
            else:
                data_buckets[pid] = [p]

        for k in data_buckets.keys():
            data_buckets[k] = np.asarray(data_buckets[k])

        iter_list = []
        for k in data_buckets.keys():
            np.random.shuffle(data_buckets[k])
            iter_list += (np.array_split(data_buckets[k]
                           , int(data_buckets[k].shape[0]/self.batch_size)))
        shuffle(iter_list) # shuffle all the batches so they arent ordered by bucket
        # size
        for i in iter_list: 
            yield i.tolist() # as it was stored in an array
    
    def __len__(self):
        return self.data_source.shape[0]
    
    def element_to_bucket_id(self, x, seq_length):
        boundaries = list(self.bucket_boundaries)
        buckets_min = [np.iinfo(np.int32).min] + boundaries
        buckets_max = boundaries + [np.iinfo(np.int32).max]
        conditions_c = np.logical_and(
          np.less_equal(buckets_min, seq_length),
          np.less(seq_length, buckets_max))
        bucket_id = np.min(np.where(conditions_c))
        return bucket_id

In [13]:
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from transformers.models.albert import AlbertConfig, AlbertModel

class F1_Loss(nn.Module):
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        y_true = F.one_hot(y_true, -1).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)
        
        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        return 1 - f1.mean()

class CFG:
    learning_rate=1.0e-3
    batch_size=64
    num_workers=4
    print_freq=100
    test_freq=1
    start_epoch=0
    num_train_epochs=3
    warmup_steps=30
    max_grad_norm=1000
    gradient_accumulation_steps=1
    weight_decay=0.01    
    dropout=0.0
    emb_size=100
    hidden_size=160
    nlayers=2
    nheads=8
    seq_len=1200
    target_size = 7
    num_fts_len = 28
    fts_len = sum([x[1] for x in cat_embs_size]) + len(num_features)

class TransfomerModel(nn.Module):
    def __init__(self, cfg):
        super(TransfomerModel, self).__init__()
        
        self.cfg = cfg
        
        self.embeds = torch.nn.ModuleList([
            nn.Embedding(a,b) for a,b in cat_embs_size
        ])
        
        input_len = cfg.fts_len

        self.config = AlbertConfig( 
            3, # not used
            embedding_size=input_len,
            hidden_size=cfg.hidden_size,
            num_hidden_layers=cfg.nlayers,
            num_attention_heads=cfg.nheads,
            intermediate_size=cfg.hidden_size,            
            hidden_dropout_prob=cfg.dropout,
            attention_probs_dropout_prob=cfg.dropout,
            max_position_embeddings=cfg.seq_len,
            type_vocab_size=1,
            position_embedding_type = "relative_key_query"
        )        
        self.encoder = AlbertModel(self.config)        
        
        def get_reg():
            return nn.Sequential(
                nn.Linear(cfg.hidden_size + cfg.num_fts_len, cfg.hidden_size), 
                nn.LayerNorm(cfg.hidden_size),
                nn.GELU(),
                nn.Linear(cfg.hidden_size, cfg.hidden_size),
                nn.LayerNorm(cfg.hidden_size),
                nn.GELU(),
                nn.Linear(cfg.hidden_size, cfg.target_size),            
            ) 
        self.reg_layer = get_reg() 
        
    def forward(self, cat_features_vals, num_features_vals, user_features_vals, mask):        
        batch_size = cat_features_vals.size(0)
            
        seq_emb = torch.cat([m(cat_features_vals[:,:,i]) 
                             for i,m in enumerate(self.embeds)] + [num_features_vals], dim=-1)
        
        seq_length = min(self.cfg.seq_len, cat_features_vals.size(1))
        position_ids = torch.arange(seq_length, dtype=torch.long, device=cat_features_vals.device)
        position_ids = position_ids.unsqueeze(0).expand((batch_size, seq_length))
        
        encoded_layers = self.encoder(inputs_embeds=seq_emb, attention_mask=mask, position_ids=position_ids)
        sequence_output = encoded_layers[0]
        sequence_output = sequence_output[:, -1]
                
        x = torch.cat([sequence_output, user_features_vals], dim=1)
        #x = sequence_output
        
        pred_y = self.reg_layer(x)
        return pred_y
    
class AttentionWeightedAverage(nn.Module):
    def __init__(self, hidden_dim: int, return_attention: bool = False):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.return_attention = return_attention
        
        self.attention_vector = nn.Parameter(
            torch.empty(self.hidden_dim, dtype=torch.float32),
            requires_grad=True,
        )
        nn.init.xavier_normal_(self.attention_vector.unsqueeze(-1))

    def forward(
        self, x: torch.Tensor, mask: torch.Tensor
    ):
        logits = x.matmul(self.attention_vector)
        ai = (logits - logits.max()).exp()

        ai = ai * mask
        att_weights = ai / (ai.sum(dim=1, keepdim=True) + 1e-12)
        weighted_input = x * att_weights.unsqueeze(-1)
        output = weighted_input.sum(dim=1)

        if self.return_attention:
            return output, att_weights
        else:
            return output, None
    
class GRUModel(nn.Module):
    def __init__(self, cfg):
        super(GRUModel, self).__init__()
        
        self.cfg = cfg
        
        self.embeds = torch.nn.ModuleList([
            nn.Embedding(a,b) for a,b in cat_embs_size
        ])
        
        input_len = cfg.fts_len

        self.encoder = nn.GRU(input_size = cfg.fts_len,
                              hidden_size = cfg.hidden_size,
                              num_layers = cfg.nlayers,
                              batch_first = True,
                              bidirectional = True
                             )    
        #self.atn = AttentionWeightedAverage(2*cfg.hidden_size)
        
        def get_reg():
            return nn.Sequential(
                nn.Linear(4*cfg.hidden_size + cfg.num_fts_len, cfg.hidden_size), # 
                nn.LayerNorm(cfg.hidden_size),
                nn.Mish(),
                #nn.Linear(cfg.hidden_size * 2, cfg.hidden_size),
                #nn.LayerNorm(cfg.hidden_size),
                #nn.GELU(),
                nn.Linear(cfg.hidden_size, cfg.target_size),            
            ) 
        self.reg_layer = get_reg() 
        
    def forward(self, cat_features_vals, num_features_vals, user_features_vals, mask):        
        batch_size = cat_features_vals.size(0)
            
        seq_emb = torch.cat([m(cat_features_vals[:,:,i]) 
                             for i,m in enumerate(self.embeds)] + [num_features_vals], dim=-1)
        
        _, sequence_output = self.encoder(seq_emb)
        #out, _ = self.atn(out, mask)
        sequence_output = sequence_output.transpose(0,1).flatten(start_dim=1)
                
        x = torch.cat([sequence_output, user_features_vals], dim=1) #
        #x = sequence_output
        
        pred_y = self.reg_layer(x)
        return pred_y
    
class GRUBaseModel(nn.Module):
    def __init__(self, cfg):
        super(GRUBaseModel, self).__init__()
        
        self.cfg = cfg
        
        self.embeds = torch.nn.ModuleList([
            nn.Embedding(a,b) for a,b in cat_embs_size
        ])
        
        input_len = cfg.fts_len

        self.encoder = nn.GRU(input_size = cfg.fts_len,
                              hidden_size = cfg.hidden_size,
                              num_layers = cfg.nlayers,
                              batch_first = True,
                              bidirectional = True
                             )            
        def get_reg():
            return nn.Sequential(
                nn.Linear(4*cfg.hidden_size, cfg.hidden_size), 
                nn.LayerNorm(cfg.hidden_size),
                nn.Mish()           
            ) 
        self.reg_layer = get_reg()
        self.male_age = nn.Linear(cfg.hidden_size, 7)
        self.unsuper  = nn.Linear(cfg.hidden_size, 23761)
        
    def forward(self, cat_features_vals, num_features_vals):        
        batch_size = cat_features_vals.size(0)
            
        seq_emb = torch.cat([m(cat_features_vals[:,:,i]) 
                             for i,m in enumerate(self.embeds)] + [num_features_vals], dim=-1)
        
        _, sequence_output = self.encoder(seq_emb)
        sequence_output = sequence_output.transpose(0,1).flatten(start_dim=1)
        x = sequence_output
        emb = self.reg_layer(x)
        
        male_age = self.male_age(emb)
        unsuper = self.unsuper(emb)
        
        return male_age, unsuper

In [14]:
q.loc[(q.is_male == 'NA'), 'is_male'] = '0.5'
q['is_male'] = q['is_male'].astype(float)

In [15]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score 

kf = KFold(n_splits=5, shuffle=True, random_state=239)
train_users = df_target.loc[(~df_target.is_male.isnull()) & (~df_target.age.isnull())].user_id.values
ifold = 0
best_metrics = []
for tr,va in kf.split(train_users):
    print(np.isnan(user_features[train_users[tr].tolist()]).any())
    print(np.isnan(user_features[train_users[va].tolist()]).any())
    print(np.isnan(user_features[df_test.user_id.values.tolist()]).any())

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [16]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score 
import copy

kf = KFold(n_splits=5, shuffle=True, random_state=239)
train_users = df_target.loc[(~df_target.is_male.isnull()) & (~df_target.age.isnull())].user_id.values
ifold = 0
best_metrics = []
for tr,va in kf.split(train_users):
    train_df = q.loc[q.user_id.isin(train_users[tr].tolist())].reset_index(drop=True)
    val_df = q.loc[q.user_id.isin(train_users[va].tolist()) & (q.is_male != 0.5)].reset_index(drop=True)
    ds_train = AlphaDataset(train_df, is_train = True)
    ds_val = AlphaDataset(val_df)
    #train_dl = DataLoader(ds_train, batch_size=64, collate_fn=collate_batch, 
    #                      shuffle=True, num_workers=4)
    bs=256
    sampler = BySequenceLengthSampler(train_df, bs)
    train_dl = DataLoader(ds_train, batch_size=1, 
                            batch_sampler=sampler, 
                            num_workers=4,
                            collate_fn=collate_batch,
                            drop_last=False, 
                            pin_memory=False)
    val_dl = DataLoader(ds_val, batch_size=bs, collate_fn=collate_batch, 
                        shuffle=False, num_workers=4)

    #model = TransfomerModel(CFG).cuda()
    model = GRUModel(CFG)
    base_model = GRUBaseModel(CFG)
    base_model.load_state_dict(torch.load('./ckps_mms/base_model_20_best_st.pth'))
    embeds = copy.deepcopy(base_model.embeds.state_dict())
    encoder = copy.deepcopy(base_model.encoder.state_dict())
    model.embeds.load_state_dict(embeds)
    model.encoder.load_state_dict(encoder)
    model = model.cuda()
    loss1 = torch.nn.CrossEntropyLoss()
    loss2 = torch.nn.BCEWithLogitsLoss()
    
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-4},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0}
        ]

    optimizer = torch.optim.AdamW(
                                 optimizer_grouped_parameters,
                                 lr=3e-4,
                                 weight_decay=CFG.weight_decay, 
                                 eps=1e-6
                                 )
    #optimizer = torch.optim.Adam(model.parameters(), lr=CFG.learning_rate)
    #optimizer = Lion(model.parameters(), lr=CFG.learning_rate, weight_decay=CFG.weight_decay)
    
    nepochs = 20
    steps_per_epoch = 1 + len(ds_train) // 48
    nsteps = nepochs * len(ds_train) // 48
    warmup_steps = 100 # nepochs * nsteps // 10

    best_metric = 0
    for epoch in range(7):
        model.train()
        optimizer.zero_grad()
        i, l1s, l2s = 0, [], []
        for x in train_dl:
            out = model(x[0].long().cuda(), x[1].float().cuda(), 
                        x[2].float().cuda(), x[3].long().cuda())
            l2 = loss2(out[:,0], x[4][:,1].float().cuda())
            l1 = loss1(out[:,1:], x[4][:,0].long().cuda())
            loss = l1 + l2
            #print(l1.item(), l2.item())
            l1s.append(l1.item())
            l2s.append(l2.item())
            loss.backward()
            optimizer.step() 
            optimizer.zero_grad()
                        
        model.eval()
        with torch.no_grad():
            outs, targets = [],[]
            for x in val_dl:
                out = model(x[0].long().cuda(), x[1].float().cuda(), 
                            x[2].float().cuda(), x[3].long().cuda())
                outs.append(out.detach().cpu().numpy())
                targets.append(x[4].detach().cpu().numpy())
        outs = np.concatenate(outs, axis=0)
        targets = np.concatenate(targets, axis=0)

        outs[:,1] = np.argmax(outs[:,1:], axis = 1)
        f1_part = f1_score(targets[:,0], outs[:,1], average='weighted')
        gini_part = roc_auc_score(targets[:,1].astype(int), outs[:,0])
        
        new_metric = 2.0*f1_part + (2.0*gini_part - 1.0)
        print(epoch, f1_part, gini_part, new_metric, np.mean(l1s), np.mean(l2s))
        if new_metric > best_metric:
            best_metric = new_metric
            torch.save(model.state_dict(), f'./ckps_mms/msh_nn_st6{ifold}.pth')
    best_metrics.append(best_metric)
    ifold += 1

0 0.4761851213736685 0.8857521193907403 1.7238744815288176 1.21263790002891 0.4441967477755887
1 0.47354970515897177 0.887224638033483 1.7215486863849094 1.1834414508371127 0.4288298679604417
2 0.47957659257284685 0.8870159708632048 1.7331851268721032 1.163975460685435 0.4199622470175936
3 0.4753371332086234 0.8849986212685743 1.7206715089543954 1.1439578240116437 0.4100656324199268
4 0.47364065132021455 0.8852021610851907 1.7176856248108106 1.120853003788562 0.4009921420897756
5 0.4645048433642026 0.881993736909196 1.6929971605467973 1.0973371110501744 0.3912365782651163
6 0.4594827227367275 0.8798271995965972 1.6786198446666494 1.0692280298897199 0.38002529383770056
0 0.47987979909306333 0.8897663702606027 1.739292338707332 1.2180881300376536 0.4485029490544574
2 0.4832967715739721 0.8911643193952573 1.7489221819384588 1.1668649534194737 0.4210154066364634
3 0.48206863787601284 0.8875619073616531 1.7392610904753318 1.1476665741646204 0.4110946015483724
4 0.4812870752340485 0.88568259

In [17]:
np.mean(best_metrics), best_metrics

(1.7420958364531693,
 [1.7331851268721032,
  1.7489221819384588,
  1.7412916639611191,
  1.7491434121074514,
  1.7379367973867144])

In [18]:
def np_sigmoid(x):
    return 1/(1 + np.exp(-x))

def test_collate_batch(batch):
    #cat_features_vals, num_features_vals, user_features_vals, mask, targets = batch
    max_len = 0
    for c in batch:
        if c[0].shape[0] > max_len:
            max_len = c[0].shape[0]
            
    if max_len > 1200:
        max_len = 1200

    return torch.stack([
        pad_matrix(m[0], max_len) for m in batch
    ]), torch.stack([
        pad_matrix(m[1], max_len) for m in batch
    ]), torch.stack([
        torch.tensor(m[2]) for m in batch
    ]), torch.stack([
        pad_matrix(m[3], max_len) for m in batch
    ])
      
total_outs = np.zeros((50,len(ds_test),7))

for i in range(5):
    model.load_state_dict(torch.load(f'./ckps_mms/msh_nn_st6{i}.pth'))
    model.eval()

    for j in tqdm.tqdm(range(10)):
        ds_test = AlphaDataset(df_test, is_train=True)
        test_dl = DataLoader(ds_test, batch_size=256, 
                             collate_fn=test_collate_batch, 
                             shuffle=False, num_workers=4)
        with torch.no_grad():
            outs = []
            for x in test_dl:
                out = model(x[0].long().cuda(), x[1].float().cuda(), 
                            x[2].float().cuda(), x[3].long().cuda())
                outs.append(out.detach().cpu().numpy())
        outs = np.concatenate(outs, axis=0)
        total_outs[i*10+j] = outs
    
outs = np.mean(total_outs, axis=0)

df_test['age'] = np.argmax(outs[:,1:], axis = 1) + 1
df_test['is_male'] = np_sigmoid(outs[:,0])

100%|██████████| 10/10 [12:19<00:00, 73.96s/it]


In [19]:
df_test.loc[df_test[['user_id','age','is_male']].isnull().any(axis=1)]

,user_id,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,...,age_bucket_4_proba_top,age_bucket_5_proba_top,age_bucket_6_proba_top,is_male_proba_bigrams,age_bucket_1_proba_bigrams,age_bucket_2_proba_bigrams,age_bucket_3_proba_bigrams,age_bucket_4_proba_bigrams,age_bucket_5_proba_bigrams,age_bucket_6_proba_bigrams


In [20]:
df_test[['user_id','age','is_male']].head()

,user_id,age,is_male
0,6,2,0.231073
1,7,2,0.907542
2,9,1,0.070465
3,10,3,0.025942
4,11,5,0.918913


In [21]:
df_test['age'].value_counts()

2    51676
3    50783
4    17464
1    13458
5    11161
6      182
Name: age, dtype: int64

In [22]:
df_test[['user_id','age','is_male']].to_csv('nn_st6.csv', index=False)